# <center> <span style="color:#f6f794"> 🧼 The purpose of this jupyter notebook is to create pipelines to clean raw data </span> </center>

- I'll check here if the pipelines of cleaning have sense so it can be put in a .py file and use this way in the architecture of the project
- Also, Some metrics will be done in order to know what to extract and give to the client
- Furthermore feature enginering will be made

_____

### <span style="color:#f6f794"> 📚 Importing libraries and setting displays </span>

In [5]:
import pandas as pd
import numpy as np
import requests
import gzip
from io import BytesIO
import os
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import time
import csv
import re
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import re
import unicodedata

In [6]:
from sklearn.impute import SimpleImputer

### <span style="color:#f6f794"> 📂 Opening files and create dataframes of them </span>

In [7]:
# Setting paths
current_dir = os.getcwd()                            # \sentiment-analyzer-inc\eda
print(current_dir)
parent_dir = os.path.dirname(current_dir)            # \sentiment-analyzer-inc
print(parent_dir)
data_dir = os.path.join(current_dir, "data")         # \sentiment-analyzer-inc\eda\data
print(data_dir)
cleaning_file_path = os.path.join(current_dir, "")   # \sentiment-analyzer-inc\eda\data\
print(cleaning_file_path)

c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda
c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc
c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda\data
c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda\


In [8]:
GITHUB_CLEAN_URL = "https://raw.githubusercontent.com/Yael-Parra/sentiment-analyzer-inc/feature/eda/etl/data/youtube_comments_-4GmbBoYQjE.csv.gz"

def load_airbnb_data_from_github(url):
    """
    Downloading and processing Airbnb data from GitHub repository.
    """
    print("⏯️ Downloading data from GitHub...")

    try:
        # Download the compressed CSV file from GitHub
        response = requests.get(url)
        response.raise_for_status()

        print("⏯️ bar_chart: Processing compressed file..")

        # Decompressing and reading the CSV file
        with gzip.open(BytesIO(response.content), 'rt', encoding='utf-8') as f:
            df = pd.read_csv(
                f,
                sep=',',                  # Specify the delimiter
                encoding='utf-8',         # Specify the encoding
                quoting=csv.QUOTE_MINIMAL # Specify the quoting behavior
                )

        print(f"✅ white_check_mark: Data downloaded succesfully!")
        
        return df

    except requests.exceptions.RequestException as req_err:
        print(f"⚠️ Network/Request Error while downloading data: {req_err}")
        return None
    except Exception as e:
        print(f"⚠️ General Error while processing data: {e}")
        return None

# Creating dataframe from GitHub URL
df_youtube_extract = load_airbnb_data_from_github(GITHUB_CLEAN_URL)

⏯️ Downloading data from GitHub...
⏯️ bar_chart: Processing compressed file..
✅ white_check_mark: Data downloaded succesfully!


___________
# <center> <span style="color:#f6f794">  🧹🧹🧹🧹🧹🧹🧹 START CLEANING 🧹🧹🧹🧹🧹🧹🧹 </center>
__________

In [9]:
df_youtube_extract.columns

Index(['threadId', 'commentId', 'videoId', 'author', 'authorChannelId',
       'isReply', 'parentCommentId', 'publishedAtComment', 'text',
       'likeCountComment', 'replyCount'],
      dtype='object')

In [10]:
df_youtube_extract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73998 entries, 0 to 73997
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   threadId            73998 non-null  object 
 1   commentId           73998 non-null  object 
 2   videoId             73998 non-null  object 
 3   author              73968 non-null  object 
 4   authorChannelId     73998 non-null  object 
 5   isReply             73998 non-null  bool   
 6   parentCommentId     2375 non-null   object 
 7   publishedAtComment  73998 non-null  object 
 8   text                73869 non-null  object 
 9   likeCountComment    73998 non-null  int64  
 10  replyCount          71623 non-null  float64
dtypes: bool(1), float64(1), int64(1), object(8)
memory usage: 5.7+ MB


In [11]:
df_youtube_extract.sample(10)

,threadId,commentId,videoId,author,authorChannelId,isReply,parentCommentId,publishedAtComment,text,likeCountComment,replyCount
731,UgzqV8Q7Qz333CXgUtl4AaABAg,UgzqV8Q7Qz333CXgUtl4AaABAg,-4GmbBoYQjE,@sonalipatel6319,UCTTmpRk6mn02KeqtRxkTeWg,False,NaN,2025-06-20T23:16:46Z,Help,0,0.0
23297,Ugxa99Z42HbFIa5Ab4V4AaABAg,Ugxa99Z42HbFIa5Ab4V4AaABAg,-4GmbBoYQjE,@tiktocartoons,UCYGh4ftNCW7P9mrrwTKgLaQ,False,NaN,2025-05-13T21:44:22Z,Thanks for video😅🫶🏻,1,0.0
5835,UgzZXqclBkRFueod6Zh4AaABAg,UgzZXqclBkRFueod6Zh4AaABAg,-4GmbBoYQjE,@MdMurtuja-nd7ku,UCFIyWoe7CwXi83lQ0HGxPJA,False,NaN,2025-05-24T09:54:02Z,Mrbeast aapko Palestinian ka madad karna chahie,0,0.0
51912,UgyDNTC58a6WH1b28q14AaABAg,UgyDNTC58a6WH1b28q14AaABAg,-4GmbBoYQjE,@paolagarcía-v6h,UC1PJ32zvcUL44X4wGhLd1Eg,False,NaN,2025-05-10T19:59:48Z,ojala los feastbleast lleguen a argentina,0,0.0
37089,UgyPaTfmzyNXtkfsXvl4AaABAg,UgyPaTfmzyNXtkfsXvl4AaABAg,-4GmbBoYQjE,@onestep161gmailcom,UCBRuRdCjhiCFJOmw71RsmTA,False,NaN,2025-05-11T12:49:52Z,Wowowowo,1,0.0
55827,Ugy9LHmrrSSPX8svYQN4AaABAg,Ugy9LHmrrSSPX8svYQN4AaABAg,-4GmbBoYQjE,@يحيىسيف-ر5ق,UCR7boz7NKv5KrrGBXGjDgHw,False,NaN,2025-05-10T18:13:35Z,………………,0,0.0
71396,UgwOqNun_kT87Iab4U54AaABAg,UgwOqNun_kT87Iab4U54AaABAg,-4GmbBoYQjE,@Dj_pzza,UCqrFRlFaTP1TQ1z5geOy9NA,False,NaN,2025-05-10T16:08:09Z,Oo,1,0.0
36423,UgxSO_0FI0bjQoiN7st4AaABAg,UgxSO_0FI0bjQoiN7st4AaABAg,-4GmbBoYQjE,@Essaie011,UCauWkPOjdH3Nh-lqKv-e9dA,False,NaN,2025-05-11T13:57:20Z,For the next vidéo 1 gorilla vs 100 hundred fi...,0,0.0
53070,Ugyj5v8Re_p72c7dc614AaABAg,Ugyj5v8Re_p72c7dc614AaABAg,-4GmbBoYQjE,@Flnzih,UCrx14HndC5dhLBu5ESsJm4w,False,NaN,2025-05-10T19:19:09Z,Прикольно,0,0.0
17471,Ugwi0Jp3ogWdbToYqLN4AaABAg,Ugwi0Jp3ogWdbToYqLN4AaABAg,-4GmbBoYQjE,@NguyễnTín-g9w,UC5A-91SuspHDzNNhRmxEPSQ,False,NaN,2025-05-15T22:52:36Z,❤ thật là 1 video tuyệt vời của đời tôi,0,0.0


In [12]:
df_youtube_extract.shape # (73998, 11)

(73998, 11)

#### <span style="color:#F3A006"> 1️⃣ Columns </span>

- Column's names format: lowercase with underscore as spaces

In [13]:
# Changin the names to lowercase and with a underscore to as spaces to make it easier to work with.
df_youtube_extract.columns = (
    df_youtube_extract.columns
    .str.replace(r'(?<!^)(?=[A-Z])', '_', regex=True)  # Insert underscore before each uppercase (except start)
    .str.lower()                                       # Convert all to lowercase
    .str.strip()                                       # Remove leading and trailing spaces in case there were any
)

In [14]:
df_youtube_extract.columns

Index(['thread_id', 'comment_id', 'video_id', 'author', 'author_channel_id',
       'is_reply', 'parent_comment_id', 'published_at_comment', 'text',
       'like_count_comment', 'reply_count'],
      dtype='object')

#### <span style="color:#F3A006"> 2️⃣ Duplicates </span>

- There are no duplicate columns

In [15]:
# Checking if there are duplicate columns
df_youtube_extract.columns.duplicated().any()         # np.False_
df_youtube_extract["comment_id"].duplicated().any()   # np.True_
df_youtube_extract["comment_id"].nunique()            # 73992
# df_youtube_extract["comment_id"].nunique()          # 73992
# df_youtube_extract["video_id"].duplicated().any()   # np.True_
# df_youtube_extract["video_id"].nunique()            # 1

73992

In [16]:
def handle_duplicates(df):
    """
    Handle duplicate rows in the dataset with updated column names.
    """
    df_clean = df.copy()
    
    # First, identify exact duplicates across all columns
    df_clean = df_youtube_extract.drop_duplicates()
    
    # Ensure each comment_id is unique
    df_clean = df_youtube_extract.drop_duplicates(subset=['comment_id'], keep='first')
    
    # Remove likely duplicates (same author, text, and timestamp)
    df_clean = df_youtube_extract.drop_duplicates(
        subset=['author', 'text', 'published_at_comment'], 
        keep='first'
    )
    
    return df_clean

# Handling duplicates in the dataset
df_youtube_extract = handle_duplicates(df_youtube_extract)


#### <span style="color:#F3A006">  3️⃣  Nulls </span>

- Before imputing:
    - parent_comment_id:  qty of nulls:  973
    - text:               qty of nulls:    2
    - reply_count:        qty of nulls:   27

- After imputation:
    - parent_comment_id:
        - 
    - text:
        - 
    - reply_count: 
        - This value only makes sense for top-level comments, as replies do not receive further replies. Using -1 clearly distinguishes replies from root comments without mixing them with valid counts (like 0). It signals “not applicable” without being confused with an actual reply count.
     


In [17]:
# Checking if there are nulls
df_youtube_extract.isnull().sum()            # "author" : 30  |  "parent_comment_id":  71623     |  "text": 129      |    reply_count: 2375
# Checking if there are empty rows
df_youtube_extract.isna().sum()              # "author" : 30  |  "parent_comment_id":  71623     |  "text": 129      |    reply_count: 2375
# # Checkirng if there are empty strings
df_youtube_extract.eq('').sum()              # 0
# Checking if there are "nan" values
df_youtube_extract.eq('nan').sum()           # 0

thread_id               0
comment_id              0
video_id                0
author                  0
author_channel_id       0
is_reply                0
parent_comment_id       0
published_at_comment    0
text                    0
like_count_comment      0
reply_count             0
dtype: int64

In [21]:
def analyze_nulls(df):
    """
    Analyze null values percentage and provide basic recommendations
    on imputing or dropping, column by column.
    """
    total_rows = len(df)
    report = []

    for col in df.columns:
        null_count = df[col].isna().sum()
        null_pct = null_count / total_rows * 100
        
        if null_count == 0:
            recommendation = "No missing values"
        else:
            # Basic heuristic recommendations
            if col in ['thread_id', 'comment_id', 'video_id', 'text']:
                recommendation = "Drop rows with nulls (IDs or text)"
            elif col in ['author', 'author_channel_id']:
                if null_pct < 5:
                    recommendation = "Impute with 'unknown' or similar"
                else:
                    recommendation = "High missing rate: consider data quality issues"
            elif col == 'is_reply':
                recommendation = "Impute with False if missing means no reply"
            elif col == 'parent_comment_id':
                recommendation = "Keep nulls (not a reply indicator)"
            elif col == 'published_at_comment':
                if null_pct < 1:
                    recommendation = "Drop rows with null timestamps"
                else:
                    recommendation = "Consider flag + placeholder or investigate further"
            elif col in ['like_count_comment', 'reply_count']:
                recommendation = "Impute with 0 (absence = zero engagement)"
            else:
                recommendation = "Check context; impute or drop accordingly"
        
        report.append({
            "column": col,
            "null_count": null_count,
            "null_percentage": round(null_pct, 2),
            "recommendation": recommendation
        })
    
    return report

# Calling the function to analyze nulls
null_analysis_report = analyze_nulls(df_youtube_extract)
null_analysis_report

[{'column': 'thread_id',
  'null_count': np.int64(0),
  'null_percentage': np.float64(0.0),
  'recommendation': 'No missing values'},
 {'column': 'comment_id',
  'null_count': np.int64(0),
  'null_percentage': np.float64(0.0),
  'recommendation': 'No missing values'},
 {'column': 'video_id',
  'null_count': np.int64(0),
  'null_percentage': np.float64(0.0),
  'recommendation': 'No missing values'},
 {'column': 'author',
  'null_count': np.int64(30),
  'null_percentage': np.float64(0.04),
  'recommendation': "Impute with 'unknown' or similar"},
 {'column': 'author_channel_id',
  'null_count': np.int64(0),
  'null_percentage': np.float64(0.0),
  'recommendation': 'No missing values'},
 {'column': 'is_reply',
  'null_count': np.int64(0),
  'null_percentage': np.float64(0.0),
  'recommendation': 'No missing values'},
 {'column': 'parent_comment_id',
  'null_count': np.int64(71585),
  'null_percentage': np.float64(96.8),
  'recommendation': 'Keep nulls (not a reply indicator)'},
 {'column':

In [22]:
def handle_nulls(df):
    
    df_clean = df.copy()

    null_handling_strategies = {
        # IDs - must exist, drop rows if null
        'thread_id': {'action': 'drop'},
        'comment_id': {'action': 'drop'},
        'video_id': {'action': 'drop'},

        # Author info - fill with 'unknown', but track frequency externally
        'author': {'action': 'fill', 'value': 'unknown'},
        'author_channel_id': {'action': 'fill', 'value': 'unknown'},

        # Boolean flag - assume False if missing, but validate assumption
        'is_reply': {'action': 'fill', 'value': False},

        # Parent comment ID - missing means no reply, leave as is
        'parent_comment_id': {'action': 'leave'},

        # Timestamp - drop if missing values are few, otherwise impute and flag
        'published_at_comment': {'action': 'conditional_drop', 'threshold': 0.05},

        # Text - no text means no analysis, drop such rows
        'text': {'action': 'drop'},

        # Engagement metrics - fill missing values with zero
        'like_count_comment': {'action': 'fill', 'value': 0},
        'reply_count': {'action': 'fill', 'value': 0}
    }

    for column, strategy in null_handling_strategies.items():
        if column not in df_clean.columns:
            continue

        action = strategy['action']
        if action == 'drop':
            # Drop rows where this column is null
            df_clean = df_clean.dropna(subset=[column])
        elif action == 'fill':
            # Fill nulls with specified value
            df_clean[column] = df_clean[column].fillna(strategy['value'])
        elif action == 'leave':
            # Do nothing for this column
            continue
        elif action == 'conditional_drop':
            # Calculate ratio of nulls in this column
            null_ratio = df_clean[column].isna().mean()
            threshold = strategy.get('threshold', 0)
            if null_ratio <= threshold:
                # If nulls are under threshold, drop those rows
                df_clean = df_clean.dropna(subset=[column])
            else:
                # If many nulls, fill with a placeholder and add a flag column
                fill_value = pd.Timestamp('1970-01-01')  # placeholder date
                df_clean[column + '_was_null'] = df_clean[column].isna()
                df_clean[column] = df_clean[column].fillna(fill_value)

    return df_clean

In [23]:
# Handling nulls in the dataset
df_youtube_extract = handle_nulls(df_youtube_extract)

#### <span style="color:#F3A006"> 4️⃣ Changing the data types </span>

- DateTime: 
    - published_at

In [24]:
df_youtube_extract.dtypes

thread_id                object
comment_id               object
video_id                 object
author                   object
author_channel_id        object
is_reply                   bool
parent_comment_id        object
published_at_comment     object
text                     object
like_count_comment        int64
reply_count             float64
dtype: object

In [25]:
df_youtube_extract.head(2)

,thread_id,comment_id,video_id,author,author_channel_id,is_reply,parent_comment_id,published_at_comment,text,like_count_comment,reply_count
0,Ugy0E7NVmoLzOdofHcF4AaABAg,Ugy0E7NVmoLzOdofHcF4AaABAg,-4GmbBoYQjE,@MrBeast,UCX6OQ3DkcsbYNE6H8uQQuVA,False,NaN,2025-05-10T16:17:59Z,"SUBSCRIBE TO WIN $50,000! To celebrate my birt...",65127,638.0
1,Ugy0E7NVmoLzOdofHcF4AaABAg,Ugy0E7NVmoLzOdofHcF4AaABAg.AHxBl1q6T3rAHxBmoK2NNH,-4GmbBoYQjE,@2005hondaaccord-i4x,UCg2JAkEAQwbZMwYKAX43Qrg,True,Ugy0E7NVmoLzOdofHcF4AaABAg,2025-05-10T16:18:14Z,First,781,0.0


In [26]:
def convert_data_types(df, conversions):
   
    df_copy = df.copy()
    
    for col, conversion in conversions.items():
        if col not in df_copy.columns:
            continue
        try:
            if callable(conversion):
                df_copy[col] = conversion(df_copy[col])
            else:
                df_copy[col] = df_copy[col].astype(conversion)
        except Exception as e:
            print(f"Warning: Could not convert column '{col}': {e}")
    
    return df_copy

In [27]:
conversions = {
    'published_at_comment': lambda col: pd.to_datetime(col, errors='coerce'),
    'like_count_comment': 'int',
    'reply_count': 'int',
    'is_reply': 'bool',
    'author_channel_id': 'string'
}

df_youtube_extract = convert_data_types(df_youtube_extract, conversions)


#### <span style="color:#F3A006"> 5️⃣ Eliminating invisible row breakes </span>

In [31]:
# Eliminating row breaks and extra spaces in the comments
def remove_line_breaks(df, columns):
    
    df_copy = df.copy()
    
    for col in columns:
        if col in df_copy.columns:
            df_copy[col] = df_copy[col].str.replace(r'[\r\n]+', ' ', regex=True).str.strip()
    
    return df_copy

# Calling the function to remove line breaks
df_youtube_extract = remove_line_breaks(df_youtube_extract, ['text'])

#### <span style="color:#F3A006"> 6️⃣ Feature Enginering </span>

['comment_length'] 
['word_count']

In [32]:
def create_colums_lentgh(df):
    df['comment_length'] = df['text'].apply(lambda x: len(str(x)))    # How many characters in the comment
    df['word_count'] = df['text'].apply(lambda x: len(str(x).split())) # How many words in the comment

# Call function
create_colums_lentgh(df_youtube_extract)

['has_emoji', 'has_url', 'has_special_char']

In [33]:
# Checking for emojis in column 'text' since it might indicate sentiment and influence the model
EMOJI_PATTERN = re.compile(
    "[\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"   # symbols & pictographs
    "\U0001F680-\U0001F6FF"   # transport & map symbols
    "\U0001F1E0-\U0001F1FF"   # flags (iOS)
    "\U00002702-\U000027B0"   # Dingbats
    "\U000024C2-\U0001F251"   # Enclosed characters
    "]+", flags=re.UNICODE
)

# Checking for URLs in column 'text' since it might indicate sentiment and influence the model
URL_PATTERN = re.compile(r'https?://\S+|www\.\S+')

# Checking for special characters in column 'text' since it might indicate sentiment and influence the model
SPECIAL_CHAR_PATTERN = re.compile(r'[^a-zA-Z0-9\s.,?!]')

# Function to flag features in 'text'
def extract_text_features(df):
    df['has_emoji'] = df['text'].apply(lambda x: bool(EMOJI_PATTERN.search(str(x))))
    df['has_url'] = df['text'].apply(lambda x: bool(URL_PATTERN.search(str(x))))
    df['has_special_char'] = df['text'].apply(lambda x: bool(SPECIAL_CHAR_PATTERN.search(str(x))))
    return df

df_youtube_extract = extract_text_features(df_youtube_extract)


['extracted_urls']

- Removing URL's from the text column since it makes noise for the model

In [34]:
def extract_and_remove_urls(df, text_column='text'):
    """
    1. Extracts URLs to new column 'extracted_urls' (as lists)
    2. Removes URLs from original text column
    3. Adds 'has_url' boolean flag
    """
    df = df.copy()
    
    # Extract URLs (store as list)
    df['extracted_urls'] = df[text_column].apply(
        lambda x: re.findall(r'https?://\S+|www\.\S+', str(x))
    )
    
    # Remove URLs from text (keep everything else)
    df[text_column] = df[text_column].apply(
        lambda x: re.sub(r'https?://\S+|www\.\S+', '', str(x))
    )
    
    # Add simple boolean flag
    df['has_url'] = df['extracted_urls'].apply(lambda x: len(x) > 0)
    
    return df

df_youtube_extract = extract_and_remove_urls(df_youtube_extract)

['is_self_promotional']

In [35]:
self_promo_keywords = {
    # English
    'en': [
        'check out my', 'subscribe to', 'follow me', 'visit my', 'link in bio',
        'watch my', 'support my', 'my channel', 'my content', 'please subscribe',
        'giveaway on my', 'join my', 'don’t forget to follow', 'my new video',
        'check my', 'like and subscribe', 'hit subscribe', 'sub to my',
        'drop a sub', 'my latest video', 'my socials', 'follow my', 'my page',
        'my profile', 'my website', 'my blog', 'my podcast', 'my merch'
    ],
    
    # Spanish
    'es': [
        'mira mi', 'suscríbete a', 'sígueme', 'visita mi', 'enlace en bio',
        've mi', 'apoya mi', 'mi canal', 'mi contenido', 'por favor suscríbete',
        'sorteo en mi', 'únete a mi', 'no olvides seguir', 'mi nuevo video',
        'checa mi', 'dale like y suscríbete', 'suscríbete a mi', 'mis redes'
    ],
    
    # Hindi
    'hi': [
        'मेरा चैनल देखो', 'सब्सक्राइब करो', 'मुझे फॉलो करो', 'मेरी वेबसाइट देखो',
        'मेरा लिंक', 'मेरा वीडियो देखो', 'मेरे चैनल को सपोर्ट करो', 'मेरा कंटेंट',
        'कृपया सब्सक्राइब करें', 'मेरे चैनल से जुड़ें', 'मेरा नया वीडियो'
    ],
    
    # Portuguese
    'pt': [
        'confira meu', 'inscreva-se no', 'me siga', 'visite meu', 'link na bio',
        'assista meu', 'apoie meu', 'meu canal', 'meu conteúdo', 'por favor se inscreva',
        'sorteio no meu', 'junte-se ao meu', 'não esqueça de seguir'
    ],
    
    # French
    'fr': [
        'regarde mon', 'abonne-toi à', 'suis-moi', 'visite mon', 'lien en bio',
        'regarde ma', 'soutiens mon', 'ma chaîne', 'mon contenu', 'abonne-toi s\'il te plaît',
        'concours sur mon', 'rejoins mon', 'n\'oublie pas de suivre'
    ],
    
    # German
    'de': [
        'schau dir mein', 'abonniere', 'folge mir', 'besuche mein', 'link in bio',
        'sieh dir mein', 'unterstütze mein', 'mein kanal', 'mein inhalt', 'bitte abonnieren',
        'gewinnspiel auf mein', 'tritt mein bei', 'vergiss nicht zu folgen'
    ],
    
    # Russian
    'ru': [
        'посмотри мой', 'подпишись на', 'подпишись на меня', 'зайди на мой', 'ссылка в профиле',
        'посмотри мое', 'поддержи мой', 'мой канал', 'мой контент', 'пожалуйста подпишись',
        'конкурс на моем', 'присоединяйся к моему', 'не забудь подписаться'
    ],
    
    # Japanese
    'ja': [
        '私のチャンネルを見て', '登録して', 'フォローして', '私のサイトを見て',
        'プロフィールのリンク', '私の動画を見て', '私のチャンネルをサポートして', '私のコンテンツ',
        'チャンネル登録お願いします', '私の新しい動画'
    ],
    
    # Arabic
    'ar': [
        'شاهد قناتي', 'اشترك في', 'تابعني', 'زور موقعي', 'الرابط في البايو',
        'ادعم قناتي', 'قناتي', 'محتواي', 'من فضلك اشترك', 'انضم إلى قناتي',
        'لا تنسى المتابعة'
    ]
}

def is_self_promotional(text):
    if not isinstance(text, str):
        return False
        
    text = text.lower()
    # Check all languages
    for lang_keywords in self_promo_keywords.values():
        if any(phrase in text for phrase in lang_keywords):
            return True
    return False

df_youtube_extract['is_self_promotional'] = df_youtube_extract['text'].apply(is_self_promotional)

['contains_tag'] 

In [36]:
# Making a new column to detect wether the comment is tagging an account in the comment 
def contains_tag(text):
    return '@' in text

df_youtube_extract['contains_tag'] = df_youtube_extract['text'].apply(contains_tag)

df['sentiment_score'] = sentiment_results['compound']
    df['sentiment_type'] = sentiment_results['type']
    df['sentiment_intensity'] = sentiment_results['intensity']

In [42]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def analyze_sentiment(df, text_column='text'):
        
    analyzer = SentimentIntensityAnalyzer()
    
    def get_sentiment(text):
        if not isinstance(text, str) or not text.strip():
            return {'compound': 0, 'type': 'neutral', 'intensity': 'weak'}
        
        scores = analyzer.polarity_scores(text)
        compound = scores['compound']
        
        # Determine sentiment type
        if compound >= 0.05:
            sentiment = 'positive'
        elif compound <= -0.05:
            sentiment = 'negative'
        else:
            sentiment = 'neutral'
        
        # Determine intensity
        abs_compound = abs(compound)
        if abs_compound > 0.5:
            intensity = 'strong'
        elif abs_compound > 0.2:
            intensity = 'moderate'
        else:
            intensity = 'weak'
        
        return {
            'compound': compound,
            'type': sentiment,
            'intensity': intensity
        }
    
    # Apply sentiment analysis
    sentiment_results = df[text_column].apply(get_sentiment).apply(pd.Series)
    
    # Add new columns to dataframe
    df['sentiment_score'] = sentiment_results['compound']
    df['sentiment_type'] = sentiment_results['type']
    df['sentiment_intensity'] = sentiment_results['intensity']
    
    return df

# Apply to your dataframe
df_youtube_extract = analyze_sentiment(df_youtube_extract)

## <span style="color:#F3A006"> 🔟 Summary </span>

In [43]:
df_youtube_extract.columns

Index(['thread_id', 'comment_id', 'video_id', 'author', 'author_channel_id',
       'is_reply', 'parent_comment_id', 'published_at_comment', 'text',
       'like_count_comment', 'reply_count', 'comment_length', 'word_count',
       'has_emoji', 'has_url', 'has_special_char', 'extracted_urls',
       'is_self_promotional', 'contains_tag', 'sentiment_score',
       'sentiment_type', 'sentiment_intensity'],
      dtype='object')

In [44]:
df_youtube_extract

,thread_id,comment_id,video_id,author,author_channel_id,is_reply,parent_comment_id,published_at_comment,text,like_count_comment,...,word_count,has_emoji,has_url,has_special_char,extracted_urls,is_self_promotional,contains_tag,sentiment_score,sentiment_type,sentiment_intensity
0,Ugy0E7NVmoLzOdofHcF4AaABAg,Ugy0E7NVmoLzOdofHcF4AaABAg,-4GmbBoYQjE,@MrBeast,UCX6OQ3DkcsbYNE6H8uQQuVA,False,NaN,2025-05-10 16:17:59+00:00,"SUBSCRIBE TO WIN $50,000! To celebrate my birt...",65127,...,51,False,False,True,[],True,True,0.9723,positive,strong
1,Ugy0E7NVmoLzOdofHcF4AaABAg,Ugy0E7NVmoLzOdofHcF4AaABAg.AHxBl1q6T3rAHxBmoK2NNH,-4GmbBoYQjE,@2005hondaaccord-i4x,UCg2JAkEAQwbZMwYKAX43Qrg,True,Ugy0E7NVmoLzOdofHcF4AaABAg,2025-05-10 16:18:14+00:00,First,781,...,1,False,False,False,[],False,False,0.0000,neutral,weak
2,Ugy0E7NVmoLzOdofHcF4AaABAg,Ugy0E7NVmoLzOdofHcF4AaABAg.AHxBl1q6T3rAHxBp6bpcSn,-4GmbBoYQjE,@zkgamer0075,UCg66bWmg82zYX7anmjYJP9Q,True,Ugy0E7NVmoLzOdofHcF4AaABAg,2025-05-10 16:18:33+00:00,Ok,341,...,1,False,False,False,[],False,False,0.2960,positive,moderate
3,Ugy0E7NVmoLzOdofHcF4AaABAg,Ugy0E7NVmoLzOdofHcF4AaABAg.AHxBl1q6T3rAHxBpEdvOSs,-4GmbBoYQjE,@حيدر-حيدر313,UCuUyL_mW_vgqM58NAMLAqQA,True,Ugy0E7NVmoLzOdofHcF4AaABAg,2025-05-10 16:18:34+00:00,قبل 28 ثانيه,246,...,3,False,False,True,[],False,False,0.0000,neutral,weak
4,Ugy0E7NVmoLzOdofHcF4AaABAg,Ugy0E7NVmoLzOdofHcF4AaABAg.AHxBl1q6T3rAHxBpdWOXYa,-4GmbBoYQjE,@Disaster33,UC9x_0ZvmkrO9a3_NN4cRWGg,True,Ugy0E7NVmoLzOdofHcF4AaABAg,2025-05-10 16:18:37+00:00,Wow😮,155,...,1,True,False,True,[],False,False,0.5859,positive,strong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73993,Ugwqa8VVuQ3yr7JJBzh4AaABAg,Ugwqa8VVuQ3yr7JJBzh4AaABAg,-4GmbBoYQjE,@lol0ajr,UCmkmEMuEALvFktHBkSdLQaA,False,NaN,2025-05-10 16:00:12+00:00,1st😚😙💕,32,...,1,True,False,True,[],False,False,0.9403,positive,strong
73994,Ugwqa8VVuQ3yr7JJBzh4AaABAg,Ugwqa8VVuQ3yr7JJBzh4AaABAg.AHx9ihJHjDCAHxABg3f0NS,-4GmbBoYQjE,@anniesbeautyandfashionblog1381,UCfN7lJnyjb429fG5DJVH6-A,True,Ugwqa8VVuQ3yr7JJBzh4AaABAg,2025-05-10 16:04:17+00:00,Your actually,4,...,2,False,False,False,[],False,False,0.0000,neutral,weak
73995,Ugwqa8VVuQ3yr7JJBzh4AaABAg,Ugwqa8VVuQ3yr7JJBzh4AaABAg.AHx9ihJHjDCAHxADq3HMBx,-4GmbBoYQjE,@IIIIIIIIiiizx,UC8qiWnKwklmZwZkqD6OkbjQ,True,Ugwqa8VVuQ3yr7JJBzh4AaABAg,2025-05-10 16:04:35+00:00,Wow so fast ❤,5,...,4,True,False,True,[],False,False,0.8505,positive,strong
73996,Ugwqa8VVuQ3yr7JJBzh4AaABAg,Ugwqa8VVuQ3yr7JJBzh4AaABAg.AHx9ihJHjDCAHxC0J9u6Bt,-4GmbBoYQjE,@albertoorellana8796,UCtwRoU0Sd1vJGUM7vN82R5g,True,Ugwqa8VVuQ3yr7JJBzh4AaABAg,2025-05-10 16:20:13+00:00,👋🙌🙏🙌🙏👏,3,...,1,True,False,True,[],False,False,0.4939,positive,moderate


| Column Name          | Description                                                                                         |
|----------------------|-----------------------------------------------------------------------------------------------------|
| thread_id            | Unique identifier for the comment thread, grouping the original comment and its replies             |
| comment_id           | Unique identifier for each individual comment                                                       |
| video_id             | Unique identifier of the YouTube video where the comment was posted                                 |
| author               | Display name of the user who posted the comment                                                    |
| author_channel_id    | Unique identifier of the YouTube channel of the comment author                                      |
| is_reply             | Boolean indicating if the comment is a reply to another comment (True) or a top-level comment (False)|
| parent_comment_id    | Identifier of the parent comment if this comment is a reply; null or empty if top-level comment     |
| published_at_comment | Timestamp when the comment was originally published                                                 |
| text                 | The textual content of the comment                                                                 |
| like_count_comment   | Number of likes that the comment has received                                                      |
| reply_count          | Number of replies the comment has (only applicable for top-level comments)                          |


_______
# <center> <span style="color:#f6f794"> 💾💾💾💾💾 Saving in GitHub Compressed 💾💾💾💾💾</center>

In [41]:
# # Saving data in cleaning_file_path and in GitHub so it can be used in the next steps and the rest of the team can access it
cleaning_file_path = os.path.join(data_dir, "youtube_extract_cleaned.csv.gz")

# Saving the cleaned DataFrame to a compressed CSV file
df_youtube_extract.to_csv(
    cleaning_file_path,
    index=False,
    compression='gzip',
    sep=',',                  # Crucial: Specify delimiter even for compressed
    encoding='utf-8',         # Crucial: Specify encoding
    quoting=csv.QUOTE_MINIMAL # Crucial: Specify quoting behavior
)
print(f"✅ Data saved successfully at {cleaning_file_path}")

✅ Data saved successfully at c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda\data\youtube_extract_cleaned.csv.gz


In [39]:
# Saving the csv file in the data directory
cleaning_file_path = os.path.join(data_dir, "youtube_dataset_checking.csv")
# Saving the cleaned DataFrame to a CSV file
df_youtube_extract.to_csv(
    cleaning_file_path,
    index=False,
    sep=',',                  # Crucial: Specify delimiter
    encoding='utf-8',         # Crucial: Specify encoding
    quoting=csv.QUOTE_MINIMAL # Crucial: Specify quoting behavior
)
print(f"✅ Data saved successfully at {cleaning_file_path}")

✅ Data saved successfully at c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda\data\youtube_dataset_checking.csv
